<a href="https://colab.research.google.com/github/mongoq/thesis/blob/main/Classic_Motorway_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://towardsdatascience.com/implementing-real-time-object-detection-system-using-pytorch-and-opencv-70bac41148f7

In [55]:
! pip install pafy youtube-dl==2020.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
import cv2
import pafy
import youtube_dl
import torch
from time import time
import numpy as np
from torch import hub

In [93]:
class Detect():
      #  Read this for classs magic:
      #  https://cito.github.io/byte_of_python/read/class-and-object-vars.html

      # Define Youtube videos 
      #_URL = "https://www.youtube.com/watch?v=wqctLW0Hb_0" # motorway traffic
      _URL = "https://www.youtube.com/watch?v=2sGTPMTaEcA" # mein schöner benz
      _video_file = "motorway.mp4" # output file

        #"""
        #Load Yolo V5 Model and init the object
        #"""
      def __init__(self):
        self.modeltorch = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)
        #pass

      # Define Video stream
      def get_video_stream(self):
        play = pafy.new(self._URL).streams[-1] #'-1' means read the lowest quality of video.
        assert play is not None # we want to make sure there is a input to read.
        stream = cv2.VideoCapture(play.url) #create a opencv video stream.
        return stream

        """
        The function below identifies the device which is availabe to make the prediction and uses it to load and infer the frame. Once it has results it will extract the labels and cordinates(Along with scores) for each object detected in the frame.
        """
      #def score_frame(frame, model):
      def score_frame(self, frame):
        device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
        #print("score_frame: modeltorch: " + str(type(self.modeltorch)))
        self.modeltorch.to(device)
        #frame = [torch.tensor(frame)]
        #print("Frame Typ vor Numpy" + str(type(frame)) )
        #--- > frame=[np.array(frame)]
        #print("Frame Typ nach Numpy" + str(type(frame)) )
        #print("score_frame - frame Typ: " + str(type(frame)))
        #frame = torch.tensor(frame)
        #frame = [int(torch.tensor(frame)).long()]
        #print("score_frame frame " + str(frame))
        results = self.modeltorch(frame)
        labels = results.xyxyn[0][:, -1].numpy()
        cord = results.xyxyn[0][:, :-1].numpy()
        print("labels: " + str(labels))
        print("cord: " + str(cord))
        return labels, cord

        """
        The function below takes the results and the frame as input and plots boxes over all the objects which have a score higer than our threshold.
        """
      def plot_boxes(self, results, frame):
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
          row = cord[i]
          # If score is less than 0.2 we avoid making a prediction.
          if row[4] < 0.2: 
            continue
          x1 = int(row[0]*x_shape)
          y1 = int(row[1]*y_shape)
          x2 = int(row[2]*x_shape)
          y2 = int(row[3]*y_shape)
          bgr = (0, 255, 0) # color of the box
          classes = self.modeltorch.names # Get the name of label index
          label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
          cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
          cv2.putText(frame,\
                    classes[labels[i]], \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
          return frame

        """
        The Function below oracestrates the entire operation and performs the real-time parsing for video stream.
        """
      def __call__(self):
        out_file= self._video_file
        player = self.get_video_stream() #Get your video stream.
        assert player.isOpened() # Make sure that their is a stream. 
        #Below code creates a new video writer object to write our
        #output stream.
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG") #Using MJPEG codex
        out = cv2.VideoWriter(out_file, four_cc, 20, \
                          (x_shape, y_shape)) 
        ret, frame = player.read() # Read the first frame.
        while ret: # Run until stream is out of frames
          start_time = time() # We would like to measure the FPS.
          results = self.score_frame(frame) # Score the Frame
          frame = self.plot_boxes(results, frame) # Plot the boxes.
          end_time = time()
          fps = 1/np.round(end_time - start_time, 3) #Measure the FPS.
          print(f"Frames Per Second : {fps}")
          out.write(frame) # Write the frame onto the output.
          ret, frame = player.read() # Read next frame.

In [94]:
Motorway=Detect()
Motorway()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-20 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
 [    0.68714    0.067784     0.83153     0.95288     0.82212]
 [   0.023939    0.045743     0.16282     0.98662     0.72795]
 [     0.6153     0.10299     0.72777     0.90292     0.70554]
 [    0.21118     0.14434     0.28485     0.60498     0.60854]
 [    0.27823      0.1215     0.40709     0.65557     0.58999]
 [    0.56629    0.058149     0.67196     0.54147     0.49627]
 [     0.3872     0.13203     0.54357     0.60127     0.42547]
 [    0.27115    0.093693     0.33302     0.30642     0.38501]
 [    0.52536      0.5926       0.688     0.99465     0.29123]]
Frames Per Second : 5.434782608695652
labels: [          0           0           0           0           0           0           0           0           0          16          28]
cord: [[    0.79445    0.056479     0.99513     0.99546     0.91733]
 [    0.68713    0.067947     0.83182     0.95365     0.80928]
 [   0.024609    0.038106     0.15843     0.98708    

In [95]:
from google.colab import files
files.download(Detect._video_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>